In [2]:
import pandas as pd
# Setting up dataframe
df = pd.read_csv("../../assets/companylist.csv")
df

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
0,YI,"111, Inc.",8.9900,$733.08M,2018.0,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi,NaN
1,PIH,"1347 Property Insurance Holdings, Inc.",5.8000,$34.87M,2014.0,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih,NaN
2,PIHPP,"1347 Property Insurance Holdings, Inc.",25.3000,$17.71M,NaN,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pihpp,NaN
3,TURN,180 Degree Capital Corp.,1.9800,$61.62M,NaN,Finance,Finance/Investors Services,https://www.nasdaq.com/symbol/turn,NaN
4,FLWS,"1-800 FLOWERS.COM, Inc.",18.7700,$1.21B,1999.0,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws,NaN
5,BCOW,"1895 Bancorp of Wisconsin, Inc.",9.4043,$45.86M,2019.0,Finance,Banks,https://www.nasdaq.com/symbol/bcow,NaN
6,FCCY,1st Constitution Bancorp (NJ),18.8600,$162.72M,NaN,Finance,Savings Institutions,https://www.nasdaq.com/symbol/fccy,NaN
7,SRCE,1st Source Corporation,46.2700,$1.19B,NaN,Finance,Major Banks,https://www.nasdaq.com/symbol/srce,NaN
8,VNET,"21Vianet Group, Inc.",8.9900,$1.01B,2011.0,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/vnet,NaN
9,TWOU,"2U, Inc.",40.9700,$2.4B,2014.0,Technology,Computer Software: Prepackaged Software,https://www.nasdaq.com/symbol/twou,NaN
